In [2]:
import os
import pickle
import email_read_util

## Download 2007 TREC Public Spam Corpus
1. Read the "Agreement for use" https://plg.uwaterloo.ca/~gvcormac/treccorpus07/
2. Download 255 MB Corpus (trec07p.tgz) and untar into the 'chapter1/datasets' directory
3. Check that the below paths for 'DATA_DIR' and 'LABELS_FILE' exist

In [3]:
DATA_DIR = './datasets/trec07p/data/'
LABELS_FILE = './datasets/trec07p/full/index'
TRAINING_SET_RATIO = 0.7

In [4]:
labels = {}
spam_words = set()
ham_words = set()

In [ ]:
# Read the labels 
with open(LABELS_FILE) as f:
    for line in f:
        line = line.strip()